In [1]:
import os
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

Data is from [NSIDC](https://nsidc.org/data/nsidc-0046/versions/4#anchor-data-access-tools).

In [2]:
# Settings
BASE_URL = "https://daacdata.apps.nsidc.org/pub/DATASETS/nsidc0046_weekly_snow_seaice/data/"
OUTPUT_DIR = "nsidc0046_data"

# Years and months you want
TARGET_YEARS = list(range(1980, 2023))
TARGET_MONTHS = [3, 4, 5]

# Make output directory if needed
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
# Set up session with .netrc authentication
session = requests.Session()
session.auth = requests.auth.HTTPBasicAuth('', '')  # Credentials will be read from .netrc

In [5]:
# Get the HTML listing
print("Fetching file list...")
response = session.get(BASE_URL)
soup = BeautifulSoup(response.text, 'html.parser')

# Extract all .bin file links
links = soup.find_all('a', href=True)
bin_files = [link['href'] for link in links if link['href'].endswith('.bin')]

# Compile regex to extract year and month from filename
filename_pattern = re.compile(r'EASE2_N25km\.snowice\.(\d{4})(\d{2})(\d{2})-\d{8}\.v\d+\.bin')

# Filter files by year/month
filtered_files = []
for fname in bin_files:
    match = filename_pattern.match(fname)
    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        if year in TARGET_YEARS and month in TARGET_MONTHS:
            filtered_files.append(fname)

print(f"Found {len(filtered_files)} files matching your criteria.")

# Download each file
for fname in filtered_files:
    url = urljoin(BASE_URL, fname)
    output_path = os.path.join(OUTPUT_DIR, fname)

    if os.path.exists(output_path):
        print(f"Already downloaded: {fname}")
        continue

    print(f"Downloading {fname}...")
    with session.get(url, stream=True) as r:
        r.raise_for_status()
        with open(output_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

print("Download complete.")

Fetching file list...
Found 1130 files matching your criteria.
Already downloaded: EASE2_N25km.snowice.19800303-19800309.v04.bin
Already downloaded: EASE2_N25km.snowice.19800310-19800316.v04.bin
Already downloaded: EASE2_N25km.snowice.19800317-19800323.v04.bin
Already downloaded: EASE2_N25km.snowice.19800324-19800330.v04.bin
Already downloaded: EASE2_N25km.snowice.19800331-19800406.v04.bin
Already downloaded: EASE2_N25km.snowice.19800407-19800413.v04.bin
Already downloaded: EASE2_N25km.snowice.19800414-19800420.v04.bin
Already downloaded: EASE2_N25km.snowice.19800421-19800427.v04.bin
Already downloaded: EASE2_N25km.snowice.19800428-19800504.v04.bin
Already downloaded: EASE2_N25km.snowice.19800505-19800511.v04.bin
Already downloaded: EASE2_N25km.snowice.19800512-19800518.v04.bin
Already downloaded: EASE2_N25km.snowice.19800519-19800525.v04.bin
Already downloaded: EASE2_N25km.snowice.19800526-19800601.v04.bin
Already downloaded: EASE2_N25km.snowice.19810302-19810308.v04.bin
Already downl